## 3. Boyutta Bernstein-Vazirani Algoritması 

In [1]:
import cirq
from cmath import exp
from math import pi

import numpy as np
from cirq import protocols

In [2]:
imag = complex(0, 1)
A = 2 * pi * imag / 3
B = 4 * pi * imag / 3

Unitaries = [
    [
        [np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]], dtype=complex)],
        [np.array([[0, 0, 1], [1, 0, 0], [0, 1, 0]], dtype=complex)],
        [np.array([[0, 1, 0], [0, 0, 1], [1, 0, 0]], dtype=complex)]
    ],
    [
        [np.array([[1, 0, 0], [0, A, 0], [0, 0, B]], dtype=complex)],
        [np.array([[0, 0, B], [1, 0, 0], [0, A, 0]], dtype=complex)],
        [np.array([[0, A, 0], [0, 0, B], [1, 0, 0]], dtype=complex)]
    ],
    [
        [np.array([[1, 0, 0], [0, B, 0], [0, 0, A]], dtype=complex)],
        [np.array([[0, 0, A], [1, 0, 0], [0, B, 0]], dtype=complex)],
        [np.array([[0, B, 0], [0, 0, A], [1, 0, 0]], dtype=complex)]
    ]
]

In [3]:
class CustomUGate(cirq.Gate):
    def __init__(self, unitary_matrix):
        self.unitary_matrix = unitary_matrix
        super().__init__()

    def _qid_shape_(self):
        return 3,
    
    def _num_qubits_():
        return 1

    def _unitary_(self):
        return self.unitary_matrix

    @staticmethod
    def _circuit_diagram_info_(args):
        return 'U'

    @property
    def transform_matrix(self) -> np.ndarray:
        return self._unitary_()

    def __str__(self):
        return str(self._unitary_())

In [4]:
class QutritIdle(cirq.Gate):
    def _qid_shape_(self):
        return 3,

    @staticmethod
    def _unitary_():
        return Unitaries[0][0][0]

    @staticmethod
    def _circuit_diagram_info_(args):
        return 'I'

    @property
    def transform_matrix(self) -> np.ndarray:
        return self._unitary_()

    def __str__(self):
        return str(self._unitary_())

In [5]:
class QutritHadamard(cirq.Gate):
    def _qid_shape_(self):
        return 3,

    @staticmethod
    def _unitary_():
        arr = np.array([
            [1, 1, 1],
            [1, exp(A), exp(B)],
            [1, exp(B), exp(A)]
        ], dtype=complex)
        arr *= 1 / pow(3, 0.5)
        return arr

    @staticmethod
    def _circuit_diagram_info_(args):
        return 'H'

    @property
    def transform_matrix(self) -> np.ndarray:
        return self._unitary_()

    def __str__(self):
        return str(self._unitary_())

In [6]:
class QutritHadamardHermitik(cirq.Gate):
    def _qid_shape_(self):
        return 3,

    @staticmethod
    def _unitary_():
        return QutritHadamard().transform_matrix.conjugate().T

    @staticmethod
    def _circuit_diagram_info_(args):
        return 'Ht'

    @property
    def transform_matrix(self) -> np.ndarray:
        return self._unitary_()

    def __str__(self):
        return str(self._unitary_())

In [7]:
class QutritCNOT(cirq.Gate):
    def _qid_shape_(self):
        return 3, 3,
    
    def _num_qubits_(self):
        return 2

    @staticmethod
    def _unitary_():
        arr = np.array([
            [1, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 1, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 1, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 1, 0, 0, 0],
            [0, 0, 0, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 1, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 1, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 1],
            [0, 0, 0, 0, 0, 0, 1, 0, 0]
        ], dtype=complex)
        return arr

    @property
    def transform_matrix(self) -> np.ndarray:
        return self._unitary_()

    @staticmethod
    def _circuit_diagram_info_(args):
        return protocols.CircuitDiagramInfo(
            wire_symbols=('@', 'X'))

    def __str__(self):
        return str(self._unitary_())

In [8]:
class QutritCNOTHermitik(cirq.Gate):
    def _qid_shape_(self):
        return 3, 3,
    
    def _num_qubits_():
        return 2

    @staticmethod
    def _unitary_():
        return QutritCNOT().transform_matrix.conjugate().T

    @staticmethod
    def _circuit_diagram_info_(args):
        return protocols.CircuitDiagramInfo(
            wire_symbols=('@\'', 'X'))

    @property
    def transform_matrix(self) -> np.ndarray:
        return self._unitary_()

    def __str__(self):
        return str(self._unitary_())

In [9]:
class ControlledQutritPhaseGate(cirq.Gate):
    def __init__(self, root):
        self._root = root
        
    def _qid_shape_(self):
        return 3, 3, 

    def _unitary_(self):
        arr = np.array([
            [1, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 1, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 1, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, exp(A / self._root), 0, 0, 0, 0],
            [0, 0, 0, 0, 0, exp(B / self._root), 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 1, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, exp(A / self._root), 0],
            [0, 0, 0, 0, 0, 0, 0, 0, exp(B / self._root)]
        ], dtype=complex)
        return arr

    @staticmethod
    def _circuit_diagram_info_(args):
        return protocols.CircuitDiagramInfo(
            wire_symbols=('@', 'Z'))

    @property
    def transform_matrix(self) -> np.ndarray:
        return self._unitary_()

    def __str__(self):
        return str(self._unitary_())

In [10]:
class QutritPhaseGate(cirq.Gate):
    def __init__(self, root):
        self._root = root
        
    def _qid_shape_(self):
        return 3, 

    def _unitary_(self):
        arr = np.array([
            [1, 0, 0],
            [0, exp(A / self._root), 0],
            [0, 0, exp(B / self._root)]
        ], dtype=complex)
        return arr

    @staticmethod
    def _circuit_diagram_info_(args):
        return "Z"

    @property
    def transform_matrix(self) -> np.ndarray:
        return self._unitary_()

    def __str__(self):
        return str(self._unitary_())

In [11]:
def test(shots, secret):
    dimension = 3
    
    # Gizli numrayı incele.
    for s in secret:
        if int(s) >= dimension:
            print("Gizli numarada içerisinde kuditlerin boyuttan büyük rakam olamaz!")
            return
    
    # Quditleri oluştur.
    qudits = cirq.LineQid.range(secret.__len__() + 1, dimension=dimension)
    last_qudit = qudits[-1]
    
    # Hepsine Hadamart uygula.
    circuit = cirq.Circuit()
    for i in range(secret.__len__()):
        circuit.append(QutritHadamard().on(qudits[i]))
    circuit.append(QutritHadamard().on(last_qudit))
    
    # Son qudite boyutun faz kapısını uygula.
    circuit.append(QutritPhaseGate(root=1).on(last_qudit))
    
    # Aktarım devresi.
    secret = secret[::-1]
    for i in range(secret.__len__()):
        if secret[i] == '0':
            pass
        elif secret[i] == '1':
            circuit.append(QutritCNOT().on(qudits[i], last_qudit))
        else:
            circuit.append(QutritCNOTHermitik().on(qudits[i], last_qudit))
    
    # Son Hadamardları ve ölçümleri ekle
    for i in range(secret.__len__()):
        circuit.append(QutritHadamard().on(qudits[i]))
    
    # Tüm qubitlere ölçüm ekle.
    for i in range(secret.__len__()):
        circuit.append(cirq.measure(qudits[i], key=str(i)))
    print(circuit)
    
    # Simule et.
    sim = cirq.Simulator()
    res = sim.run(circuit, repetitions=shots)
    return res

In [12]:
# Kodlanmak istenen gizli mesaj.
secret = "12120"
res = test(1024, secret)

result_string = ""
for i in range(secret.__len__()):
    new_list = list()
    res_list = res.measurements[str(i)].tolist()
    for r in res_list:
        new_list.append(r[0])
    res_list = np.array(new_list)
    counts = np.bincount(res_list)
    result_string += str(np.argmax(counts))

print("Gizli numara: ",  secret)
print("Bulunan ölçüm sonuçları: ", result_string[::-1])
print("Doğru mu? {}".format(secret == result_string[::-1]))

0 (d=3): ───H───H───M('0')───────────────────────────────────────────

1 (d=3): ───H───────@'───────H───M('1')──────────────────────────────
                    │
2 (d=3): ───H───────┼────────@───H────────M('2')─────────────────────
                    │        │
3 (d=3): ───H───────┼────────┼───@'───────H────────M('3')────────────
                    │        │   │
4 (d=3): ───H───────┼────────┼───┼────────@────────H────────M('4')───
                    │        │   │        │
5 (d=3): ───H───Z───X────────X───X────────X──────────────────────────
Gizli numara:  12120
Bulunan ölçüm sonuçları:  12120
Doğru mu? True


## 4. Boyutta Bernstein-Vazirani Algoritması 

In [13]:
import cirq
from cmath import exp
from math import pi
import numpy as np
from cirq import protocols

In [14]:
# complex 0 + 1i
Imag = complex(0, 1)

A = (2 * pi * Imag / 4)
B = (4 * pi * Imag / 4)
C = (6 * pi * Imag / 4)
D = (8 * pi * Imag / 4)
E = (12 * pi * Imag / 4)
F = (18 * pi * Imag / 4)

In [15]:
Unitaries = [
    [
        [np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]], dtype=complex)],
        [np.array([[0, 0, 0, 1], [1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0]], dtype=complex)],
        [np.array([[0, 0, 1, 0], [0, 0, 0, 1], [1, 0, 0, 0], [0, 1, 0, 0]], dtype=complex)],
        [np.array([[0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1], [1, 0, 0, 0]], dtype=complex)]
    ],
    [
        [np.array([[1, 0, 0, 0], [0, A, 0, 0], [0, 0, B, 0], [0, 0, 0, C]], dtype=complex)],
        [np.array([[0, 0, 0, C], [1, 0, 0, 0], [0, A, 0, 0], [0, 0, B, 0]], dtype=complex)],
        [np.array([[0, 0, B, 0], [0, 0, 0, C], [1, 0, 0, 0], [0, A, 0, 0]], dtype=complex)],
        [np.array([[0, A, 0, 0], [0, 0, B, 0], [0, 0, 0, C], [1, 0, 0, 0]], dtype=complex)]
    ],
    [
        [np.array([[1, 0, 0, 0], [0, B, 0, 0], [0, 0, D, 0], [0, 0, 0, E]], dtype=complex)],
        [np.array([[0, 0, 0, E], [1, 0, 0, 0], [0, B, 0, 0], [0, 0, D, 0]], dtype=complex)],
        [np.array([[0, 0, D, 0], [0, 0, 0, E], [1, 0, 0, 0], [0, B, 0, 0]], dtype=complex)],
        [np.array([[0, B, 0, 0], [0, 0, D, 0], [0, 0, 0, E], [1, 0, 0, 0]], dtype=complex)]

    ],
    [
        [np.array([[1, 0, 0, 0], [0, C, 0, 0], [0, 0, E, 0], [0, 0, 0, F]], dtype=complex)],
        [np.array([[0, 0, 0, F], [1, 0, 0, 0], [0, C, 0, 0], [0, 0, E, 0]], dtype=complex)],
        [np.array([[0, 0, E, 0], [0, 0, 0, F], [1, 0, 0, 0], [0, C, 0, 0]], dtype=complex)],
        [np.array([[0, C, 0, 0], [0, 0, E, 0], [0, 0, 0, F], [1, 0, 0, 0]], dtype=complex)]
    ]
]

In [16]:
class ControlledQuqritPhaseGate(cirq.Gate):
    def __init__(self, root):
        self.root = root
        
    def _qid_shape_(self):
        return 4, 4,

    def _unitary_(self):
        arr = np.zeros(shape=(16, 16), dtype=complex)
        arr[0][0], arr[1][1], arr[2][2], arr[3][3] = 1, 1, 1, 1
        arr[4][7], arr[5][4], arr[6][5], arr[7][6] = 1, 1, 1, 1
        arr[8][10], arr[9][11], arr[10][8], arr[11][9] = 1, exp(A / self.root), exp(B / self.root), exp(C / self.root)
        arr[12][13], arr[13][14], arr[14][15], arr[15][12] = 1, exp(A / self.root), exp(B / self.root), exp(C / self.root)
        return arr

    @property
    def transform_matrix(self) -> np.ndarray:
        return self._unitary_()

    def _circuit_diagram_info_(self, args):
        return protocols.CircuitDiagramInfo(
            wire_symbols=('@', 'Z^{}'.format(self.root)))

    def __str__(self):
        return str(self._unitary_())
    

class QuqritHadamard(cirq.Gate):
    def _qid_shape_(self):
        return 4,

    @staticmethod
    def _unitary_():
        arr = np.array([
            [1, 1, 1, 1],
            [1, exp(A), exp(B), exp(C)],
            [1, exp(B), exp(D), exp(E)],
            [1, exp(C), exp(E), exp(F)]
        ], dtype=complex)
        arr *= 1 / 2
        return arr

    @staticmethod
    def _circuit_diagram_info_(args):
        return 'H'

    @property
    def transform_matrix(self) -> np.ndarray:
        return self._unitary_()

    def __str__(self):
        return str(self._unitary_())


class QuqritHadamardHermitik(cirq.Gate):
    def _qid_shape_(self):
        return 4,

    @staticmethod
    def _unitary_():
        return QuqritHadamard().transform_matrix.conjugate().T

    @staticmethod
    def _circuit_diagram_info_(args):
        return 'Ht'

    @property
    def transform_matrix(self) -> np.ndarray:
        return self._unitary_()

    def __str__(self):
        return str(self._unitary_())
    
class CustomUGate(cirq.Gate):
    def __init__(self, unitary_matrix):
        self.unitary_matrix = unitary_matrix

    def _qid_shape_(self):
        return 4,

    def _unitary_(self):
        return self.unitary_matrix

    @staticmethod
    def _circuit_diagram_info_(args):
        return 'U'

    @property
    def transform_matrix(self) -> np.ndarray:
        return self._unitary_()

    def __str__(self):
        return str(self._unitary_())
    
class QuqritPhaseGate(cirq.Gate):
    def __init__(self, root):
        self._root = root
        
    def _qid_shape_(self):
        return 4, 

    def _unitary_(self):
        arr = np.array([
            [1, 0, 0, 0],
            [0, exp(A / self._root), 0, 0],
            [0, 0, exp(B / self._root), 0],
            [0, 0, 0, exp(C / self._root)]
        ], dtype=complex)
        return arr

    @staticmethod
    def _circuit_diagram_info_(args):
        return "Z"

    @property
    def transform_matrix(self) -> np.ndarray:
        return self._unitary_()

    def __str__(self):
        return str(self._unitary_())

class QuqritCNOT(cirq.Gate):
    def _qid_shape_(self):
        return 4, 4,

    @staticmethod
    def _unitary_():
        arr = np.zeros(shape=(16, 16), dtype=complex)
        arr[0][0], arr[1][1], arr[2][2], arr[3][3] = 1, 1, 1, 1
        arr[4][7], arr[5][4], arr[6][5], arr[7][6] = 1, 1, 1, 1
        arr[8][10], arr[9][11], arr[10][8], arr[11][9] = 1, 1, 1, 1
        arr[12][13], arr[13][14], arr[14][15], arr[15][12] = 1, 1, 1, 1
        return arr

    @property
    def transform_matrix(self) -> np.ndarray:
        return self._unitary_()

    @staticmethod
    def _circuit_diagram_info_(args):
        return protocols.CircuitDiagramInfo(
            wire_symbols=('@', 'X'))

    def __str__(self):
        return str(self._unitary_())


class QuqritCNOTHermitik(cirq.Gate):
    def _qid_shape_(self):
        return 4, 4,

    @staticmethod
    def _unitary_():
        return QuqritCNOT().transform_matrix.conjugate().T

    @staticmethod
    def _circuit_diagram_info_(args):
        return protocols.CircuitDiagramInfo(
            wire_symbols=('@\'', 'X'))

    @property
    def transform_matrix(self) -> np.ndarray:
        return self._unitary_()

    def __str__(self):
        return str(self._unitary_())

class QuqritCNOTHermitik_second(cirq.Gate):
    def _qid_shape_(self):
        return 4, 4,

    @staticmethod
    def _unitary_():
        arr = np.zeros(shape=(16, 16), dtype=complex)
        arr[0][0], arr[1][1], arr[2][2], arr[3][3] = 1, 1, 1, 1
        arr[4][7], arr[5][4], arr[6][5], arr[7][6] = 1, 1, 1, 1
        arr[8][9], arr[9][10], arr[10][11], arr[11][8] = 1, 1, 1, 1
        arr[12][14], arr[13][15], arr[14][12], arr[15][13] = 1, 1, 1, 1
        return arr

    @property
    def transform_matrix(self) -> np.ndarray:
        return self._unitary_()

    @staticmethod
    def _circuit_diagram_info_(args):
        return protocols.CircuitDiagramInfo(
            wire_symbols=('@', 'X'))

    def __str__(self):
        return str(self._unitary_())



In [18]:
def test(shots, secret):
    dimension = 4
    
    # Gizli numrayı incele.
    for s in secret:
        if int(s) >= dimension:
            print("Gizli numarada boyuttan büyük rakam olamaz")
            return
    
    # Quditleri oluştur.
    qudits = cirq.LineQid.range(secret.__len__() + 1, dimension=dimension)
    last_qudit = qudits[-1]
    
    # Hepsine Hadamart uygula.
    circuit = cirq.Circuit()
    for i in range(secret.__len__()):
        circuit.append(QuqritHadamard().on(qudits[i]))
    circuit.append(QuqritHadamard().on(last_qudit))
    
    # Son qudite boyutun faz kapısını uygula.
    circuit.append(QuqritPhaseGate(root=1).on(last_qudit))
    
    # Aktarım devresi.
    secret = secret[::-1]
    for i in range(secret.__len__()):
        if secret[i] == '0':
            pass
        elif secret[i] == '1':
            circuit.append(QuqritCNOT().on(qudits[i], last_qudit))
        elif secret[i] == '2':
            circuit.append(QuqritCNOTHermitik_second().on(qudits[i], last_qudit))
        else:
            circuit.append(QuqritCNOTHermitik().on(qudits[i], last_qudit))
    
    # Son Hadamardları ve ölçümleri ekle
    for i in range(secret.__len__()):
        circuit.append(QuqritHadamard().on(qudits[i]))
        
    for i in range(secret.__len__()):
        circuit.append(cirq.measure(qudits[i], key=str(i)))
    print(circuit)
    
    # Simule et.
    sim = cirq.Simulator()
    res = sim.run(circuit, repetitions=shots)
    return res

In [21]:
# Kodlanmak istenen gizli mesaj.
secret = "323211"
res = test(1024, secret)

result_string = ""
for i in range(secret.__len__()):
    new_list = list()
    res_list = res.measurements[str(i)].tolist()
    for r in res_list:
        new_list.append(r[0])
    res_list = np.array(new_list)
    counts = np.bincount(res_list)
    result_string += str(np.argmax(counts))
print("Gizli numara: ",  secret)
print("Bulunan ölçüm sonuçları: ", result_string[::-1])
print("Doğru mu? {}".format(secret == result_string[::-1]))

0 (d=4): ───H───────@───H───M('0')────────────────────────────────────────────────
                    │
1 (d=4): ───H───────┼───@───H────────M('1')───────────────────────────────────────
                    │   │
2 (d=4): ───H───────┼───┼───@────────H────────M('2')──────────────────────────────
                    │   │   │
3 (d=4): ───H───────┼───┼───┼────────@'───────H────────M('3')─────────────────────
                    │   │   │        │
4 (d=4): ───H───────┼───┼───┼────────┼────────@────────H────────M('4')────────────
                    │   │   │        │        │
5 (d=4): ───H───────┼───┼───┼────────┼────────┼────────@'───────H────────M('5')───
                    │   │   │        │        │        │
6 (d=4): ───H───Z───X───X───X────────X────────X────────X──────────────────────────
Gizli numara:  323211
Bulunan ölçüm sonuçları:  323211
Doğru mu? True
